In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Configuration
batch_size=256
epochs=100
latent_dim=64
num_samples=2979
data_path = "data/hindi.txt"

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index=dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict(
[(char,i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

Number of samples: 2773
Number of unique input tokens: 70
Number of unique output tokens: 88
Max sequence length for inputs: 89
Max sequence length for outputs: 103


In [4]:
# LSTM Model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

lstm = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

lstm.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 70)]           0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None, 88)]           0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 34560     ['input_1[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                              

In [5]:
# GRU Model
# encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
# encoder = keras.layers.GRU(latent_dim, return_state=True)
# encoder_outputs, state_h= encoder(encoder_inputs)

# encoder_states = [state_h]

# decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
# decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
# decoder_outputs = decoder_dense(decoder_outputs)

# gru = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# gru.summary()

In [6]:
lstm.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
lstm.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
history_lstm = lstm.save("lstm.keras")

Epoch 1/20
9/9 [==============================] - 3s 97ms/step - loss: 3.8860 - accuracy: 0.5062 - val_loss: 2.5921 - val_accuracy: 0.6269
Epoch 2/20
9/9 [==============================] - 0s 32ms/step - loss: 1.6384 - accuracy: 0.7683 - val_loss: 2.1550 - val_accuracy: 0.6269
Epoch 3/20
9/9 [==============================] - 0s 31ms/step - loss: 1.3384 - accuracy: 0.7683 - val_loss: 2.0116 - val_accuracy: 0.6269
Epoch 4/20
9/9 [==============================] - 0s 31ms/step - loss: 1.2560 - accuracy: 0.7683 - val_loss: 1.9519 - val_accuracy: 0.6269
Epoch 5/20
9/9 [==============================] - 0s 30ms/step - loss: 1.2119 - accuracy: 0.7683 - val_loss: 2.0772 - val_accuracy: 0.6269
Epoch 6/20
9/9 [==============================] - 0s 31ms/step - loss: 1.1601 - accuracy: 0.7683 - val_loss: 1.7596 - val_accuracy: 0.6269
Epoch 7/20
9/9 [==============================] - 0s 32ms/step - loss: 1.1113 - accuracy: 0.7683 - val_loss: 1.7981 - val_accuracy: 0.6269
Epoch 8/20
9/9 [===========

In [7]:
# gru.compile(
#     optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
# )
# gru.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.2,
# )
# # Save model
# history_gru = gru.save("gru.keras")

In [8]:
# Define sampling models for LSTM
# Define sampling models
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(([target_seq] + states_value), verbose = 0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [9]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 227ms/step
-
Input sentence: Wow!
Decoded sentence: मम                                                                                                      
1/1 [==============================] - 0s 13ms/step
-
Input sentence: Help!
Decoded sentence: मम                                                                                                      
1/1 [==============================] - 0s 14ms/step
-
Input sentence: Jump.
Decoded sentence: मम                                                                                                      
1/1 [==============================] - 0s 13ms/step
-
Input sentence: Jump.
Decoded sentence: मम                                                                                                      
1/1 [==============================] - 0s 13ms/step
-
Input sentence: Jump.
Decoded sentence: मम                                                                                                      
1/1 [